In [1]:
# Install Colab
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
# Install Stable Baselines, Gym, and Pytao
!apt-get install ffmpeg freeglut3-dev xvfb
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install gymnasium
!pip install scikit-optimize

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglu1-mesa
  libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libice-dev libopengl-dev libsm-dev
  libxfont2 libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
Suggested packages:
  libice-doc libsm-doc libxt-doc
The following NEW packages will be installed:
  freeglut3 freeglut3-dev libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libice-dev libopengl-dev
  libsm-dev libxfont2 libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 25 newly installed, 0 to remove and 0 not upgraded.
Need to

In [1]:
!conda install -c conda-forge pytao
!conda install -c conda-forge bmad

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pytao


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    cached-property-1.5.2      |       hd8ed1ab_1           4 KB  conda-forge
    cached_property-1.5.2      |     pyha770c72_1          11 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    h5py-3.11.0                |nompi_py310hf054cd7_101         1.1 MB  conda-forge
    hdf5-1.14.3                |nompi_h4f84152_100         3.7 MB  conda-forge
    libaec-1

In [5]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 918.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 31.9 MB/s eta 0:00:00


In [1]:
# Import libraries
import environ as en
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from stable_baselines3 import SAC
#from stable_baselines3 import PPO
#from stable_baselines3 import TD3
from stable_baselines3.sac.policies import MlpPolicy
#from stable_baselines3.ppo.policies import MlpPolicy
#from stable_baselines3.td3.policies import MlpPolicy
import pytao
from tqdm import tqdm

In [2]:
# Num of updates = max_steps_per_episode / num_steps_in_model

# Hyperparameters (Default)
model_type = "SAC" #############################################################
max_steps_per_episode = 100
num_steps_in_model = 64 # PPO Hyp
train_freq = 1 # SAC #Hyp
gradient_steps = 1 # SAC  #Hyp
gae_lambda = 0.95 # PPO Hyp
my_convergence_val = 0.01 ##################################################################################
my_bad_convergence_val = 0.02 # Hyp
min_step_value = 5 # Like Training convergence
my_reward_alpha = 1.0 # Hyp
my_reward_beta = 1.0 # Hyp
my_reward_fun = 4 ####################################################################
what_to_do = "x_only"


############# Hyperparameters that are tuned ##########
default_learning_rate = 0.0001 #Hyp
default_gamma = 0.99 # SAC #Hyp
default_tau = 0.005 # SAC  #Hyp
default_batch_size = 64

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
if what_to_do == "x_and_y":
  our_env = en.LTBEnv("tao.init",
                  {"orbit.x": "MW", "orbit.y": "MW"},
                  ["correctors_x", "correctors_y"],
                  -0.01,
                  0.01,
                  max_steps = max_steps_per_episode,
                  convergence_val = my_convergence_val,
                  bad_convergence_val = my_bad_convergence_val,
                  reward_alpha = my_reward_alpha,
                  reward_beta = my_reward_beta,
                  reward_fun = my_reward_fun)
elif what_to_do == "x_only":
  our_env = en.LTBEnv("tao.init",
                  {"orbit.x": "MW"},
                  ["correctors_x"],
                  -0.01,
                  0.01,
                  max_steps = max_steps_per_episode,
                  convergence_val = my_convergence_val,
                  bad_convergence_val = my_bad_convergence_val,
                  reward_alpha = my_reward_alpha,
                  reward_beta = my_reward_beta,
                  reward_fun = my_reward_fun)
elif what_to_do == "y_only":
  our_env = en.LTBEnv("tao.init",
                  {"orbit.y": "MW"},
                  ["correctors_y"],
                  -0.01,
                  0.01,
                  max_steps = max_steps_per_episode,
                  convergence_val = my_convergence_val,
                  bad_convergence_val = my_bad_convergence_val,
                  reward_alpha = my_reward_alpha,
                  reward_beta = my_reward_beta,
                  reward_fun = my_reward_fun)


In [4]:
# Define the search space for hyperparameters
search_space = [
    Real(1e-6, 1e-2, name='learning_rate'),
    Real(0.8, 0.9999, name='gamma'),
    Real(0.001, 0.2, name='tau'),
    Integer(16, 128, name='batch_size')
]

# Define a function to evaluate the model
def evaluate_model(model, env, n_eval_episodes=10):
    all_episode_rewards = []
    for _ in range(n_eval_episodes):
        episode_rewards = 0
        done = False
        obs, _ = env.reset()
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            episode_rewards += reward
        all_episode_rewards.append(episode_rewards)
    mean_reward = np.mean(all_episode_rewards)
    return mean_reward, np.max(np.abs(obs)), action

# Objective function to minimize
@use_named_args(search_space)
def objective(learning_rate, gamma, tau, batch_size):
    env = our_env  # change this to correct env
    model = SAC(MlpPolicy, env, verbose=0, train_freq = train_freq, gradient_steps = gradient_steps, learning_rate = learning_rate, batch_size = batch_size, gamma = gamma, tau = tau)
    model.learn(total_timesteps=300, progress_bar = True)
    # Evaluate the model
    mean_reward, _, _ = evaluate_model(model, env)
    return -mean_reward

In [ ]:
# Perform Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=10, random_state=0)

# Extract the best hyperparameters
best_hyperparams = {
    'learning_rate': result.x[0],
    'gamma': result.x[1],
    'tau': result.x[2],
    'batch_size': result.x[3]
}

print(f"Best hyperparameters: {best_hyperparams}")

Output()

convergence achieved, current_orbit_value = 0.0075114403

convergence achieved, current_orbit_value = 0.0095995711

convergence achieved, current_orbit_value = 0.0029529492

convergence achieved, current_orbit_value = 0.0090790389

...

convergence achieved, current_orbit_value = 0.0060173526

convergence achieved, current_orbit_value = 0.008567072

convergence achieved, current_orbit_value = 0.0072130186

convergence achieved, current_orbit_value = 0.0080775849

convergence achieved, current_orbit_value = 0.0027138022

In [16]:
# Look at performance after training for a short time
tuned_model = SAC(MlpPolicy, our_env, verbose=0, train_freq = train_freq, gradient_steps = gradient_steps, learning_rate = best_hyperparams["learning_rate"], batch_size = best_hyperparams["batch_size"], gamma = best_hyperparams["gamma"], tau = best_hyperparams["tau"])
tuned_model.learn(total_timesteps=750, progress_bar = True)
print(f"Average reward, Max Orbit, and Action For Tuned Model: {evaluate_model(tuned_model, our_env)}")

LiveError: Only one live display may be active at once

In [ ]:
base_model = SAC(MlpPolicy, our_env, verbose=0, train_freq = train_freq, gradient_steps = gradient_steps, learning_rate = default_learning_rate, batch_size = default_batch_size, gamma = default_gamma, tau = default_tau)
base_model.learn(total_timesteps=750, progress_bar = True)
print(f"Average reward, Max Orbit, and Action For Base Model: {evaluate_model(base_model, our_env)}")

In [ ]:
## ONLY FOR COMBINED ACTIONS
x_action = None
y_action = None

tao = env.tao
en.set_new_variable_values(tao, ["correctors_x"], x_action)
en.set_new_variable_values(tao, ["correctors_y"], y_action)

print(en.get_data_values(tao, {"orbit.x": "MW", "orbit.y": "MW"}))
print(f"Max Orbit: {np.max(np.abs(en.get_data_values(tao, {"orbit.x": "MW", "orbit.y": "MW"})))}")